# 从零开始的股票交易深度强化学习：使用集成策略进行多股票交易

本教程展示了如何使用 OpenAI DRL 在一个 Jupyter Notebook 中通过集成策略交易多只股票 | 展示于 ICAIF 2020

* 该笔记本是我们论文的重新实现：《用于自动化股票交易的深度强化学习：一种集成策略》，使用 FinRL 实现。
* 查看 Medium 博客以获取详细解释：[https://medium.com/@ai4finance/deep-reinforcement-learning-for-automated-stock-trading-f1dad0126a02](https://medium.com/@ai4finance/deep-reinforcement-learning-for-automated-stock-trading-f1dad0126a02)
* 请将任何问题报告到我们的 Github：[https://github.com/AI4Finance-LLC/FinRL-Library/issues](https://github.com/AI4Finance-LLC/FinRL-Library/issues)
* **Pytorch 版本**





# Content

* [1. 问题定义](#0)
* [2. 入门 - 加载 Python 包](#1)
* [2.1. 安装包](#1.1)
* [2.2. 检查附加包](#1.2)
* [2.3. 导入包](#1.3)
* [2.4. 创建文件夹](#1.4)
* [3. 下载数据](#2)
* [4. 数据预处理](#3)
* [4.1. 技术指标](#3.1)
* [4.2. 执行特征工程](#3.2)
* [5. 构建环境](#4)
* [5.1. 训练与交易数据拆分](#4.1)
* [5.2. 用户自定义环境](#4.2)
* [5.3. 初始化环境](#4.3)
* [6. 实现 DRL 算法](#5)
* [7. 回测性能](#6)
* [7.1. 回测统计](#6.1)
* [7.2. 回测绘图](#6.2)
* [7.3. 基准统计](#6.3)
* [7.4. 与股票市场指数比较](#6.4)

<a id='0'></a>
# Part 1. 问题定义

此问题旨在为单只股票交易设计一种自动化交易解决方案。我们将股票交易过程建模为马尔可夫决策过程（MDP）。然后，我们将交易目标表述为一个最大化问题。

该算法使用深度强化学习（DRL）算法进行训练，强化学习环境的组成部分包括：



* 动作：动作空间描述了智能体与环境交互时允许执行的动作。通常，动作 a ∈ A 包含三个动作：a ∈ {−1, 0, 1}，其中 -1、0、1 分别代表卖出、持有和买入一股。此外，一个动作可以作用于多股股票。我们使用动作空间 {−k， ...， −1， 0， 1， ...， k}，其中 k 表示股票的数量。例如，“买入 10 股苹果公司股票”或“卖出 10 股苹果公司股票”分别为 10 或 -10。

* 奖励函数：r(s, a, s') 是激励智能体学习更好行动的机制。当智能体处于状态 s 时采取行动 a 并到达新状态 s' 时投资组合价值的变化，即 r(s, a, s') = v' - v，其中 v' 和 v 分别表示状态 s' 和 s 下的投资组合价值。

* 状态：状态空间描述了智能体从环境中接收到的观测值。正如人类交易员在执行交易前需要分析各种信息一样，我们的交易智能体也会观察许多不同的特征，以便在交互式环境中更好地学习。

* 环境：道琼斯 30 种工业股票平均价格指数成分股



我们将用于本案例研究的单只股票数据是从雅虎财经 API 获得的。该数据包含开盘价、最高价、最低价、收盘价和成交量。


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [ ]:
# ## install finrl library
!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git



<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them.
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

<a id='1.4'></a>
## 2.4. Create Folders

In [ ]:
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [ ]:
print(DOW_30_TICKER)

In [ ]:
# TRAIN_START_DATE = '2009-04-01'
# TRAIN_END_DATE = '2021-01-01'
# TEST_START_DATE = '2021-01-01'
# TEST_END_DATE = '2022-06-01'
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.config_tickers import DOW_30_TICKER

TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TEST_START_DATE = '2021-10-01'
TEST_END_DATE = '2023-03-01'

df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = DOW_30_TICKER).fetch_data()

# 第四部分：数据预处理
数据预处理是训练高质量机器学习模型的关键步骤。我们需要检查缺失数据并进行特征工程，以便将数据转换为模型就绪的状态。

* ​**添加技术指标**​
  在实际交易中，需要考虑各种信息，例如历史股票价格、当前持仓份额、技术指标等。在本文中，我们演示了两个趋势跟踪技术指标：MACD 和 RSI。

* ​**添加波动指数**​
  风险厌恶反映了投资者是否会选择保本。它还影响一个人在面对不同市场波动水平时的交易策略。为了在最坏情况下（例如 2007-2008 年的金融危机）控制风险，FinRL 采用了衡量极端资产价格波动的金融波动指数。

In [ ]:
 INDICATORS = ['macd',
               'rsi_30',
               'cci_30',
               'dx_30']

In [ ]:
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

In [ ]:
len(state), len(data)

<a id='4'></a>
# 第五部分：设计环境
考虑到自动化股票交易任务的随机性和交互性，金融任务被建模为一个**马尔可夫决策过程（MDP）​**问题。训练过程包括观察股票价格变化、采取行动和计算奖励，以使代理能够相应地调整其策略。通过与环境的交互，交易代理将随着时间的推移推导出最大化奖励的交易策略。

我们的交易环境基于 OpenAI Gym 框架，根据时间驱动模拟的原则，使用真实市场数据模拟实时股票市场。

**动作空间**描述了代理与环境交互时允许的动作。通常，动作 a 包括三个动作：{-1, 0, 1}，其中 -1、0、1 分别表示卖出、持有和买入一股。此外，动作可以针对多股执行。我们使用动作空间 {-k,…,-1, 0, 1, …, k}，其中 k 表示买入的股票数量，-k 表示卖出的股票数量。例如，“买入 10 股 AAPL”或“卖出 10 股 AAPL”分别表示为 10 或 -10。由于策略是基于高斯分布定义的，连续动作空间需要归一化到 [-1, 1]，以确保其归一化和对称性。

In [ ]:
stock_dimension = len(processed.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


In [ ]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "print_verbosity":5

}

# buy_cost_list = sell_cost_list = [0.001] * stock_dimension
# num_stock_shares = [0] * stock_dimension
# env_kwargs = {
#     "hmax": 100,
#     "initial_amount": 1000000,
#     "num_stock_shares": num_stock_shares,
#     "buy_cost_pct": buy_cost_list,
#     "sell_cost_pct": sell_cost_list,
#     "state_space": state_space,
#     "stock_dim": stock_dimension,
#     "tech_indicator_list": INDICATORS,
#     "action_space": stock_dimension,
#     "reward_scaling": 1e-4
# }

<a id='5'></a>
# 第六部分：实现深度强化学习（DRL）算法
* DRL 算法的实现基于 ​**OpenAI Baselines**​ 和 ​**Stable Baselines**。Stable Baselines 是 OpenAI Baselines 的一个分支，进行了重大的结构重构和代码清理。
* FinRL 库包含经过微调的标准 DRL 算法，例如 DQN、DDPG、多智能体 DDPG、PPO、SAC、A2C 和 TD3。我们还允许用户通过调整这些 DRL 算法来设计自己的 DRL 算法。

* 在本笔记本中，我们使用滚动窗口集成方法（[参考代码](https://github.com/AI4Finance-LLC/Deep-Reinforcement-Learning-for-Automated-Stock-Trading-Ensemble-Strategy-ICAIF-2020/blob/80415db8fa7b2179df6bd7e81ce4fe8dbf913806/model/models.py#L92)）训练和验证 3 个智能体（A2C、PPO、DDPG）。

In [ ]:
rebalance_window = 63 # `rebalance_window` 是重新训练模型的天数。
validation_window = 63 # `validation_window` 是进行验证和交易的天数（例如，如果 `validation_window=63`，则验证和交易周期均为 63 天）。

ensemble_agent = DRLEnsembleAgent(df=processed,
                 train_period=(TRAIN_START_DATE,TRAIN_END_DATE),
                 val_test_period=(TEST_START_DATE,TEST_END_DATE),
                 rebalance_window=rebalance_window,
                 validation_window=validation_window,
                 **env_kwargs)
# e_train_gym = StockTradingEnv(df = processed, **env_kwargs)
# agent = DRLAgent(e_train_gym)
# if_using_a2c = True
# model_a2c = agent.get_model("a2c")
# # if if_using_a2c:
# #   tmp_path = RESULTS_DIR + '/a2c'
# #   new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
# #   model_a2c.set_logger(new_logger_a2c)
# trained_a2c = agent.train_model(model=model_a2c,
#                              tb_log_name='a2c',
#                              total_timesteps=50000)

In [ ]:
A2C_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.005,
                    'learning_rate': 0.0007
                    }

PPO_model_kwargs = {
                    "ent_coef":0.01,
                    "n_steps": 2048,
                    "learning_rate": 0.00025,
                    "batch_size": 128
                    }

DDPG_model_kwargs = {
                      #"action_noise":"ornstein_uhlenbeck",
                      "buffer_size": 10_000,
                      "learning_rate": 0.0005,
                      "batch_size": 64
                    }

SAC_model_kwargs = {
    "batch_size": 64,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

TD3_model_kwargs = {"batch_size": 100, "buffer_size": 1000000, "learning_rate": 0.0001}




timesteps_dict = {'a2c' : 10_000,
                 'ppo' : 10_000,
                 'ddpg' : 10_000,
                 'sac' : 10_000,
                 'td3' : 10_000
                 }

In [ ]:
df_summary = ensemble_agent.run_ensemble_strategy(A2C_model_kwargs,
                                                 PPO_model_kwargs,
                                                 DDPG_model_kwargs,
                                                 SAC_model_kwargs,
                                                 TD3_model_kwargs,
                                                 timesteps_dict)

In [ ]:
df_summary

<a id='6'></a>
# 第七部分：回测我们的策略
回测在评估交易策略表现中起着关键作用。自动化回测工具是首选，因为它可以减少人为错误。我们通常使用 Quantopian 的 pyfolio 包来回测我们的交易策略。它易于使用，并且包含各种单独的图表，这些图表提供了交易策略表现的全面视图。

In [ ]:
unique_trade_date = processed[(processed.date > TEST_START_DATE)&(processed.date <= TEST_END_DATE)].date.unique()

In [ ]:
df_trade_date = pd.DataFrame({'datadate':unique_trade_date})

df_account_value=pd.DataFrame()
for i in range(rebalance_window+validation_window, len(unique_trade_date)+1,rebalance_window):
    temp = pd.read_csv('results/account_value_trade_{}_{}.csv'.format('ensemble',i))
    df_account_value = pd.concat([df_account_value, temp], ignore_index=True)
sharpe=(252**0.5)*df_account_value.account_value.pct_change(1).mean()/df_account_value.account_value.pct_change(1).std()
print('Sharpe Ratio: ',sharpe)
df_account_value=df_account_value.join(df_trade_date[validation_window:].reset_index(drop=True))

In [ ]:
df_account_value.head()

In [ ]:
%matplotlib inline
df_account_value.account_value.plot()

<a id='6.1'></a>
## 7.1 BackTestStats
传入 `df_account_value`，这些信息存储在 `env` 类中。


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
df_dji_ = get_baseline(
        ticker="^DJI",
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(df_dji_, value_col_name = 'close')

In [ ]:
df_dji = pd.DataFrame()
df_dji['date'] = df_account_value['date']
df_dji['dji'] = df_dji_['close'] / df_dji_['close'][0] * env_kwargs["initial_amount"]
print("df_dji: ", df_dji)
df_dji.to_csv("df_dji.csv")
df_dji = df_dji.set_index(df_dji.columns[0])
print("df_dji: ", df_dji)
df_dji.to_csv("df_dji+.csv")

df_account_value.to_csv('df_account_value.csv')


<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:


# print("==============Compare to DJIA===========")
# %matplotlib inline
# # S&P 500: ^GSPC
# # Dow Jones Index: ^DJI
# # NASDAQ 100: ^NDX
# backtest_plot(df_account_value,
#               baseline_ticker = '^DJI',
#               baseline_start = df_account_value.loc[0,'date'],
#               baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])
df.to_csv("df.csv")
df_result_ensemble = pd.DataFrame({'date': df_account_value['date'], 'ensemble': df_account_value['account_value']})
df_result_ensemble = df_result_ensemble.set_index('date')

print("df_result_ensemble.columns: ", df_result_ensemble.columns)

# df_result_ensemble.drop(df_result_ensemble.columns[0], axis = 1)
print("df_trade_date: ", df_trade_date)
# df_result_ensemble['date'] = df_trade_date['datadate']
# df_result_ensemble['account_value'] = df_account_value['account_value']
df_result_ensemble.to_csv("df_result_ensemble.csv")
print("df_result_ensemble: ", df_result_ensemble)
print("==============Compare to DJIA===========")
result = pd.DataFrame()
# result = pd.merge(result, df_result_ensemble, left_index=True, right_index=True)
# result = pd.merge(result, df_dji, left_index=True, right_index=True)
result = pd.merge(df_result_ensemble, df_dji, left_index=True, right_index=True)
print("result: ", result)
result.to_csv("result.csv")
result.columns = ['ensemble', 'dji']

%matplotlib inline
plt.rcParams["figure.figsize"] = (15,5)
plt.figure();
result.plot();